## Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('data1/data_under.csv')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,3,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,3,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,3,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [4]:
df['y'].value_counts(normalize=True)

0    0.887346
1    0.112654
Name: y, dtype: float64

As from our pervoius work we find out age, loan and month dont really signifiance over our target values so we can remove from our prediction 

In [5]:
def age(dataframe):
    dataframe.loc[dataframe['age'] <= 32, 'age'] = 1
    dataframe.loc[(dataframe['age'] > 32) & (dataframe['age'] <= 47), 'age'] = 2
    dataframe.loc[(dataframe['age'] > 47) & (dataframe['age'] <= 70), 'age'] = 3
    dataframe.loc[(dataframe['age'] > 70) & (dataframe['age'] <= 98), 'age'] = 4
           
    return dataframe

age(df);

In [6]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
cat_var=['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome','y']
for i in cat_var:
    df[i]=LE.fit_transform(df[i])
    
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,2,3,1,0,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,2,7,1,3,1,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,1,7,1,3,0,2,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,1,0,1,1,0,0,0,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,2,7,1,3,0,0,2,1,6,1,...,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [7]:
X= df.iloc[:,:-1]
y= df.iloc[:,-1:]

In [8]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split
from sklearn.linear_model import LogisticRegression, Ridge,Lasso, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score,classification_report
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [9]:
from imblearn.over_sampling import SMOTE 

In [10]:
sc=StandardScaler()
sc.fit_transform(X)

array([[ 1.38823133, -0.20157925, -0.2837415 , ...,  0.88644656,
         0.71245988,  0.33167991],
       [ 1.38823133,  0.91122681, -0.2837415 , ...,  0.88644656,
         0.71245988,  0.33167991],
       [ 0.02907204,  0.91122681, -0.2837415 , ...,  0.88644656,
         0.71245988,  0.33167991],
       ...,
       [ 1.38823133,  0.35482378, -0.2837415 , ..., -2.22495344,
        -1.49518647, -2.8156966 ],
       [ 0.02907204,  1.46762984, -0.2837415 , ..., -2.22495344,
        -1.49518647, -2.8156966 ],
       [ 2.74739063,  0.35482378, -0.2837415 , ..., -2.22495344,
        -1.49518647, -2.8156966 ]])

In [11]:
sm = SMOTE(random_state = 2)
X_sm, y_sm = sm.fit_sample(X, y)

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X_sm,y_sm,test_size=0.25,random_state=2)

In [13]:
y_train.shape

(54822, 1)

In [14]:
X_train.shape

(54822, 20)

In [15]:
X_test.shape

(18274, 20)

In [16]:
?LogisticRegression

In [17]:
lr=LogisticRegression(solver = 'liblinear')
lr.fit(X_train,y_train)
pred_lr=lr.predict(X_test)
pred_tr=lr.predict(X_train)
print(confusion_matrix(y_test,pred_lr))
score_lr= accuracy_score(y_test,pred_lr)
score_tr= accuracy_score(y_train,pred_tr)
print("Accuracy Score is: ", score_lr, score_tr)
print("F1 Score is: ", f1_score(y_test,pred_lr))
print(classification_report(y_test, pred_lr))

[[8102 1086]
 [ 943 8143]]
Accuracy Score is:  0.8889679325818102 0.8910473897340484
F1 Score is:  0.8892164892164891
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      9188
           1       0.88      0.90      0.89      9086

    accuracy                           0.89     18274
   macro avg       0.89      0.89      0.89     18274
weighted avg       0.89      0.89      0.89     18274



In [18]:
lr=LogisticRegression(penalty = 'l2',solver = 'liblinear')
lr.fit(X_train,y_train)
pred_lr=lr.predict(X_test)
print(confusion_matrix(y_test,pred_lr))
score_lr= accuracy_score(y_test,pred_lr)
print("Accuracy Score is: ", score_lr)
print("F1 Score is: ", f1_score(y_test,pred_lr))
print(classification_report(y_test, pred_lr))

[[8102 1086]
 [ 943 8143]]
Accuracy Score is:  0.8889679325818102
F1 Score is:  0.8892164892164891
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      9188
           1       0.88      0.90      0.89      9086

    accuracy                           0.89     18274
   macro avg       0.89      0.89      0.89     18274
weighted avg       0.89      0.89      0.89     18274



In [19]:
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)
pred_tr=knn.predict(X_train)
pred_knn=knn.predict(X_test)
print(classification_report(y_train,pred_tr))
print(classification_report(y_test,pred_knn))
confusion_matrix(y_test,pred_knn)

              precision    recall  f1-score   support

           0       0.99      0.89      0.94     27360
           1       0.90      0.99      0.94     27462

    accuracy                           0.94     54822
   macro avg       0.95      0.94      0.94     54822
weighted avg       0.95      0.94      0.94     54822

              precision    recall  f1-score   support

           0       0.98      0.85      0.91      9188
           1       0.86      0.99      0.92      9086

    accuracy                           0.92     18274
   macro avg       0.92      0.92      0.92     18274
weighted avg       0.92      0.92      0.92     18274



array([[7786, 1402],
       [ 134, 8952]])

In [20]:
score_knn = cross_val_score(knn,y_test,pred_knn,cv=5,scoring = 'f1')
print(score_knn)
print("Mean of the cross validation scores:",score_knn.mean())

[0.92204785 0.92280072 0.92449923 0.91462471 0.92093742]
Mean of the cross validation scores: 0.9209819857757486


In [21]:
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
pred_tr=dt.predict(X_train)
pred_dt=dt.predict(X_test)
confusion_matrix(y_test,pred_dt)
confusion_matrix(y_train,pred_tr)

array([[27360,     0],
       [    0, 27462]])

In [22]:
score_dt=cross_val_score(dt,y_test,pred_dt,cv=5)
print(score_dt)
print("Mean of the cross validation scores:",score_dt.mean())

[0.91819425 0.92585499 0.92585499 0.92585499 0.93322386]
Mean of the cross validation scores: 0.9257966196368951


In [23]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
pred_rf=rf.predict(X_test)
confusion_matrix(y_test,pred_rf)

array([[8571,  617],
       [ 328, 8758]])

In [24]:
score_rf=cross_val_score(rf,y_test,pred_dt,scoring='f1',cv=5)
print(score_rf)
print("Mean of the cross validation scores:",score_rf.mean())

[0.91859515 0.92601693 0.9263787  0.9258144  0.93326039]
Mean of the cross validation scores: 0.9260131148558113


In [25]:
xgb_clf= xgb.XGBClassifier()
xgb_clf.fit(X_train,y_train)
pred_xgb=xgb_clf.predict(X_test)
confusion_matrix(y_test,pred_xgb)

[16:36:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([[8611,  577],
       [ 416, 8670]])

In [26]:
score_xgb = cross_val_score(xgb_clf,y_test,pred_xgb,scoring = 'f1',cv=5)
print(score_xgb)
print("Mean of the cross validation scores:",score_xgb.mean())

[16:36:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:36:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:36:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:36:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [27]:
parameters = {'penalty':('l1', 'l2','elasticnet','none'),'solver':('lbfgs', 'liblinear')}
lr1=LogisticRegression()
log= GridSearchCV(lr1, parameters,cv=5)
log.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'penalty': ('l1', 'l2', 'elasticnet', 'none'),
                         'solver': ('lbfgs', 'liblinear')})

In [28]:
log.best_params_

{'penalty': 'l2', 'solver': 'liblinear'}

In [29]:
pre1=log.predict(X_train)
pre=log.predict(X_test)
print(confusion_matrix(y_test,pre))

[[8102 1086]
 [ 943 8143]]


In [30]:
score_tr= accuracy_score(y_train,pre1)
score_lr= accuracy_score(y_test,pre)
print("Accuracy Score is: ", score_lr,score_tr)

Accuracy Score is:  0.8889679325818102 0.8910473897340484


In [31]:
parameters = {'n_neighbors':[1,2,3,4,5], 'leaf_size':[10,20,30,50]}
knn1=KNeighborsClassifier()
kn= GridSearchCV(knn1, parameters,cv=5)
kn.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': [10, 20, 30, 50],
                         'n_neighbors': [1, 2, 3, 4, 5]})

In [32]:
?KNeighborsClassifier

In [33]:
kn.best_params_

{'leaf_size': 10, 'n_neighbors': 1}

In [34]:
pretr=kn.predict(X_train)
pre1=kn.predict(X_test)
print(confusion_matrix(y_test,pre1))

[[8180 1008]
 [  95 8991]]


In [35]:
score_tr= accuracy_score(y_train,pretr)
score_lr= accuracy_score(y_test,pre1)
print("Accuracy Score is: ", score_lr,score_tr)

Accuracy Score is:  0.9396410200284557 1.0


In [36]:
parameters = {'criterion' : ("gini", "entropy"), 'max_depth':[3,5,7,10]}
dt1=DecisionTreeClassifier()
dt= GridSearchCV(dt1, parameters,cv=5)
dt.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': [3, 5, 7, 10]})

In [37]:
dt.best_params_

{'criterion': 'gini', 'max_depth': 10}

In [38]:
re2=dt.predict(X_train)

pre2=dt.predict(X_test)
print(confusion_matrix(y_test,pre2))
print(confusion_matrix(y_train,re2))

[[8181 1007]
 [ 551 8535]]
[[24940  2420]
 [ 1170 26292]]


In [39]:
score_tr=accuracy_score(y_train,re2)
score_lr= accuracy_score(y_test,pre2)
print("Accuracy Score is: ", score_lr,score_tr)

Accuracy Score is:  0.9147422567582357 0.9345153405567108


In [40]:
parameters = {'n_estimators':[10,50,100,150,200],'criterion' : ("gini", "entropy"), 'max_depth':[3,5,10,20,30]}
rc1=RandomForestClassifier()
rc= GridSearchCV(rc1, parameters,cv=5)
rc.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': [3, 5, 10, 20, 30],
                         'n_estimators': [10, 50, 100, 150, 200]})

In [42]:
pre2=rc.predict(X_test)
score_lr= accuracy_score(y_test,pre2)
print("Accuracy Score is: ", score_lr)

Accuracy Score is:  0.9479041260807705


In [43]:
parameters = {'n_estimators':[10,50,100,150,200]}
xg1=xgb.XGBClassifier()
xg= GridSearchCV(xg1, parameters,cv=5)
xg.fit(X_train,y_train)

[17:10:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:10:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None,

In [44]:
xg.best_params_

{'n_estimators': 200}

In [45]:
pre4=xg.predict(X_test)
print(confusion_matrix(y_test,pre4))

[[8664  524]
 [ 426 8660]]


In [46]:
score_lr= accuracy_score(y_test,pre4)
print("Accuracy Score is: ", score_lr)

Accuracy Score is:  0.9480135711940462
